# Python for AI Projects

## Introduction

**Supervised Machine Learning**

In this Jupyter notebook - we'll quickly setup our Python environment and get started with our Explore California supervised machine learning exercises.

### Practice Exercises

1. Ingest and explore our 2 datasets for product recommendation and purchase prediction
2. Implement `scikit-learn` pipelines for multi-class classification - product recommendations
3. Implement `scikit-learn` pipelines for binary classification - purchase predictions
4. Investigate model outputs and explanations using `SHAP` and `LIME`

### Getting Started

To execute each cell in this notebook - you can click on the play button on the left of each cell or hit `command/shift + enter` to run individual cells one-by-one.

In [ ]:
# Initial setup steps
# ====================

# Install Python libraries
!pip install --quiet lightgbm==4.6.0
!pip install --quiet optuna==4.4.0
!pip install --quiet lime==0.2.0.1

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259.git data

# Need to change directory into "data" to download git lfs data objects
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct
%cd ..

# Load in some Python Libraries
# =============================
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# We'll use a style that is colorblind-friendly
plt.style.use('tableau-colorblind10')

# Turn off warnings for cleaner output
warnings.filterwarnings("ignore")

# 1. Data Exploration

Let's first load in our `product_recommendation` and `purchase_prediction` datasets using Pandas.

In [ ]:
# Load in our datasets
product_recommendation_df = pd.read_csv("data/product_recommendation.csv")
purchase_prediction_df = pd.read_csv("data/purchase_prediction.csv")

## 1.1 Multi-Class Classification

Let’s start by exploring our `product_recommendation_df`. We’ll be treating this as a **multi-class classification** problem, where the goal is to predict which product a user is most likely to purchase based on their individual attributes.

## 1.1.1 Exploratory Data Analysis

Before we build any models, it’s important to take some time to understand the structure and quality of our dataset. In this section, we’ll perform a few key data exploration steps:

- ✅ Preview the first few rows to understand the data format and feature names
- 📊 Check the distribution of our target variable `product_name` to see how balanced the classes are
- 🔍 Look for any missing values or data quality issues
- 🧠 Review the number and types of features available — in this case, we’re working with binary indicators (0 or 1)

These simple checks help us catch problems early and set up our classification pipeline with confidence.

In [ ]:
# Preview dataset
product_recommendation_df

In [ ]:
# Check class distribution
product_recommendation_df['product_name'].value_counts()

In [ ]:
# Count missing values in each column
# We can sort the results to see which columns have the most missing values
product_recommendation_df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Check feature data types
product_recommendation_df.dtypes.value_counts()

In [ ]:
# Get list of feature columns (excluding ID and target)
feature_cols = product_recommendation_df.columns.difference(['user_id', 'product_name'])

# Check if all values are binary (0 or 1)
non_binary = [
    col for col in feature_cols
    if not product_recommendation_df[col].isin([0, 1]).all()
]

print("Non-binary columns:", non_binary if non_binary else "✅ All features are binary.")

### 1.1.2 Reusable EDA Function

Before training any machine learning model, it’s essential to understand your dataset. To make this easier and repeatable, we can create a reusable helper function called `explore_multiclass_dataset()`

This function performs our same key exploratory data analysis (EDA) steps as above, but this time, tailored for generic **multi-class classification problems**.

In [ ]:
def explore_multiclass_dataset(df, target_col='product_name', id_col='user_id'):
    """
    Perform basic exploratory data analysis (EDA) for a multi-class classification dataset.

    Parameters:
    ----------
    df : pd.DataFrame
        The input dataset containing a multi-class target column and feature columns.

    target_col : str, default='product_name'
        The name of the target column to predict. This should be a categorical feature with more than two unique classes.

    id_col : str, default='user_id'
        The name of a unique identifier column to exclude from binary feature validation.

    This function performs:
    ------------------------
    - A preview of the first few rows of the dataset
    - A count and bar plot of the target class distribution
    - A missing value check across all columns
    - A summary of feature data types
    - A validation that all non-ID, non-target features are binary indicators (0 or 1)
    """

    # --------------------
    # Preview the dataset
    # --------------------
    print("📌 Preview of dataset:")
    display(df.head())

    # --------------------
    # Show class distribution of the target column
    # --------------------
    print("\n📊 Target variable distribution:")
    class_counts = df[target_col].value_counts()
    display(class_counts)

    # --------------------
    # Plot class distribution bar chart
    # --------------------
    print("\n📉 Class distribution plot:")
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.bar(class_counts.index, class_counts.values, color='skyblue')
    ax.set_title("Product Class Distribution")
    ax.set_xlabel("Product Name")
    ax.set_ylabel("Number of Purchases")
    ax.set_xticks(range(len(class_counts)))
    ax.set_xticklabels(class_counts.index, rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # --------------------
    # Check for missing values
    # --------------------
    print("\n🔍 Missing values per column:")
    display(df.isnull().sum().sort_values(ascending=False))

    # --------------------
    # Show data type breakdown
    # --------------------
    print("\n🧠 Feature type summary:")
    display(df.dtypes.value_counts())

    # --------------------
    # Binary feature validation
    # --------------------
    print("\n✅ Binary feature check (excluding ID and target):")
    feature_cols = df.columns.difference([id_col, target_col])
    non_binary = [
        col for col in feature_cols
        if not df[col].isin([0, 1]).all()
    ]

    if non_binary:
        print("⚠️ Non-binary columns detected:", non_binary)
    else:
        print("✅ All features are binary (0 or 1).")


### 1.1.3 Using our EDA Function

You can also apply this function to **any dataset** that follows this structure:

- A **target column** with multiple class labels (e.g. product names, categories)
- One or more **feature columns**, ideally binary indicators (0 or 1)
- An optional **ID column** used to uniquely identify rows (not used for modeling)

In [ ]:
# Apply function for our product_recommendation_df dataset
explore_multiclass_dataset(
  product_recommendation_df,  # Input dataframe
  target_col='product_name',  # Default target column
  id_col='user_id'            # Default ID column
)

## 1.3 Binary Classification

Now let’s explore our `purchase_prediction_df`. In this case, we’re working on a **binary classification** problem, where the goal is to predict whether a user will make a **tour product purchase** based on their individual attributes.


In [ ]:
purchase_prediction_df.head()

### 1.3.1 Binary Classification EDA Function

To streamline our exploratory data analysis for binary classification problems, we’ve created a reusable Python function: `explore_binary_dataset()`.

This function helps you quickly understand the structure, balance, and quality of your dataset — and is especially useful for previewing training data before fitting a machine learning model.

Here’s what it does:

- 📌 **Preview the dataset** with the first few rows  
- 📊 **Inspect the distribution** of your binary target variable (e.g. 0 vs 1)  
- 🔄 **Break down labels by dataset split** (e.g. train, validation, test)  
- 📉 **Visualize class balance** using bar charts and stacked plots  
- 📈 **Automatically detect numeric features** and show histograms (up to 10 by default)  
- ✅ **Check binary indicator columns** for modeling readiness  
- 🔍 **Highlight missing values** and feature type summaries



In [ ]:
def explore_binary_dataset(df, target_col='label', id_col='user_id', period_order=['train', 'validation', 'test'], max_numeric=10):
    """
    Perform exploratory data analysis (EDA) for a binary classification dataset.

    Parameters:
    ----------
    df : pd.DataFrame
        The input dataset containing features, a binary target column, and optional metadata.

    target_col : str, default='label'
        The name of the binary target column (should contain only 0s and 1s).

    id_col : str, default='user_id'
        The name of the column used as a unique identifier for each row.
        This will be excluded from feature analysis.

    period_order : list of str, default=['train', 'validation', 'test']
        The desired order of dataset partitions (useful for plotting label distributions by period).

    max_numeric : int, default=10
        Maximum number of numeric (non-binary) columns to explore with summary stats and histograms.

    This function will:
    -------------------
    - Preview the first few rows of the dataset
    - Show the distribution of the binary target
    - Display label counts and proportions by `period` (if available)
    - Visualize class balance (bar charts and stacked bar plots)
    - Identify and summarize numeric (non-binary) columns
    - Limit numeric summaries to `max_numeric`
    - Identify binary columns
    - Show missing value counts per column
    """

    # --------------------
    # Dataset Preview
    # --------------------
    print("📌 Preview of dataset:")
    display(df.head())

    # --------------------
    # Target Label Distribution
    # --------------------
    print(f"\n📊 Distribution of binary target `{target_col}`:")
    target_counts = df[target_col].value_counts()
    display(target_counts)

    # --------------------
    # Label Distribution by Period
    # --------------------
    print("\n📊 Target label distribution by `period`:")
    label_by_period = (
        df.groupby("period")[target_col]
        .value_counts()
        .unstack(fill_value=0)
        .reindex(period_order)
    )
    display(label_by_period)

    print("\n📊 Proportions of each label by `period`:")
    label_proportions = label_by_period.div(label_by_period.sum(axis=1), axis=0)
    display(label_proportions)

    # --------------------
    # Plot: Overall Class Balance
    # --------------------
    print("\n📉 Overall target class distribution plot:")
    fig, ax = plt.subplots(figsize=(6, 4))
    bars = ax.bar(target_counts.index.astype(str), target_counts.values)
    ax.set_title("Binary Target Distribution")
    ax.set_xlabel("Label")
    ax.set_ylabel("Count")
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height + 200, f'{int(height)}',
                ha='center', fontsize=10)
    plt.tight_layout()
    plt.show()

    # --------------------
    # Plot: Grouped Bar Chart by Period
    # --------------------
    print("\n📊 Label breakdown by period (grouped bar chart):")
    label_by_period.plot(kind='bar', figsize=(8, 5), stacked=False)
    plt.title("Label Counts by Period")
    plt.xlabel("Period")
    plt.ylabel("Count")
    plt.xticks(rotation=0)
    plt.legend(title='Label')
    plt.tight_layout()
    plt.show()

    # --------------------
    # Plot: Normalized Stacked Bar Chart by Period
    # --------------------
    print("\n📊 Label proportions by period (normalized stacked bar chart):")
    label_proportions.plot(kind='bar', stacked=True, figsize=(8, 5))
    plt.title("Label Proportions by Period")
    plt.xlabel("Period")
    plt.ylabel("Proportion")
    plt.xticks(rotation=0)
    plt.legend(title='Label', loc='upper right')
    plt.tight_layout()
    plt.show()

    # --------------------
    # Identify Binary vs Numeric Columns
    # --------------------
    exclude_cols = [id_col, target_col, 'label_date', 'period']
    working_df = df.drop(columns=exclude_cols, errors='ignore')

    # Numeric = >2 unique numeric values
    numeric_cols = [
        col for col in working_df.select_dtypes(include=['float64', 'int64']).columns
        if df[col].nunique(dropna=True) > 2
    ]

    # Binary = all values in [0, 1] (excluding NaNs)
    binary_cols = [
        col for col in working_df.columns
        if df[col].dropna().isin([0, 1]).all()
    ]

    print("\n🔢 Feature Type Summary:")
    print(f"- Binary columns: {len(binary_cols)}")
    print(f"- Numeric (non-binary) columns: {len(numeric_cols)}")

    # --------------------
    # Plot Numeric Feature Histograms
    # --------------------
    if numeric_cols:
        print(f"\n📈 Exploring up to {max_numeric} numeric column(s):")
        for col in numeric_cols[:max_numeric]:
            print(f"\n📈 Distribution of numeric column `{col}`:")
            display(df[col].describe())
            df[col].hist(bins=30, edgecolor='black', figsize=(8, 4))
            plt.title(f"Distribution of `{col}`")
            plt.xlabel(col)
            plt.ylabel("Frequency")
            plt.grid(False)
            plt.tight_layout()
            plt.show()
        if len(numeric_cols) > max_numeric:
            print(f"⚠️ {len(numeric_cols) - max_numeric} additional numeric columns were skipped. "
                  f"Increase `max_numeric` to show more.")
    else:
        print("\nℹ️ No numeric (non-binary) features found.")

    # --------------------
    # Check for Missing Values
    # --------------------
    print("\n🔍 Missing values per column:")
    display(df.isnull().sum().sort_values(ascending=False))


### 1.3.2 Using Our Binary EDA Function

You can apply this function to your **own binary classification dataset** by passing in:

- `df`: your `pandas` DataFrame  
- `target_col`: the name of your binary label column (e.g. `'churned'`, `'clicked'`, `'purchased'`)  
- `id_col`: a unique identifier (e.g. `'user_id'`, `'session_id'`) to exclude from feature checks  
- *(optional)* `period_order`: how to order partitions like `'train'`, `'validation'`, and `'test'`  
- *(optional)* `max_numeric`: how many numeric features to explore (default = 10)

---

⚠️ If your dataset has many numeric columns, only the first 10 will be visualized. You can increase this limit using the `max_numeric` argument if needed.

This function is designed to be flexible — just make sure your target variable contains **only two values (0 and 1)**, and you’ll be ready to go!

In [ ]:
explore_binary_dataset(
  purchase_prediction_df,
  target_col='label',  # Default target column
  id_col='user_id',    # Default ID column
  period_order=['train', 'validation', 'test']  # Default period order
)

# 2. Multi-Class Classification

Now that we’ve explored our datasets, it’s time to build our first **end-to-end machine learning pipelines** using `scikit-learn`.

We’ll begin by focusing on our **multi-class classification task** — where the goal is to predict which product a user is most likely to purchase based on their binary attribute profile.


## 2.1 ML Pipelines Overview

In this section, we’ll:

- ⚙️ Build an initial pipeline that includes preprocessing and model fitting  
- 🔁 Explore how to **swap in different classifiers** (e.g. Logistic Regression, Random Forest, LightGBM)  
- 🧪 Incorporate **cross-validation and hyperparameter tuning**  
- 📊 Use evaluation tools like the **confusion matrix** and `classification_report` to assess performance  

We'll take an **incremental approach**, starting with a minimal setup and adding complexity step-by-step — so you can clearly see how each part of the ML pipeline contributes to model performance and interpretability.

Later in the tutorial, we’ll **apply a similar pipeline structure** — with a few key adjustments — to tackle our **binary classification problem**, where the goal is to predict whether a user is likely to make a purchase.



### 2.1.1 Function Scope and Reusability Tips

You might notice in the following cells that all the `scikit-learn` packages are imported *within* the function body — rather than at the top of the notebook.

This is intentional! By importing modules like `LogisticRegression` and `train_test_split` **inside the function**, we keep those dependencies **scoped only to the function itself**.

This has a few advantages:

- ✅ Keeps your global namespace clean — especially in shared or interactive notebooks  
- 📦 Makes the function more portable if you want to copy or reuse it elsewhere  
- 🧪 Helps isolate your logic during testing or modular development


### 2.1.2 Saving the Model for Deployment or Reuse

Our function implementations also returns a dictionary that includes:

- The **trained model**
- The **label encoder** (for decoding predictions)
- The train/validation splits (useful for further evaluation)
- A list of the original **feature names**

These outputs can be reused in downstream tasks such as:

- Generating predictions for new users  
- Visualizing model performance in a **dashboard**  
- Embedding the model into an **AI application or workflow**  
- Saving to disk using `joblib` or `pickle` for later deployment

For example, to save model artefacts to disk - we could run the following in a Python cell:

```python
import joblib
joblib.dump(results["model"], "logreg_model.pkl")
joblib.dump(results["label_encoder"], "label_encoder.pkl")
```

## 2.2 Logistic Regression ML Pipeline

In this step, we’ll train a simple **multiclass logistic regression model** to predict which tour product a user is most likely to purchase — based on their individual attributes.

The `train_multiclass_logistic_regression_model()` function performs the following:

- 📦 Splits the dataset into training and validation sets (using stratified sampling to preserve class balance)
- 🔠 Encodes the target labels using scikit-learn’s `LabelEncoder`
- ⚙️ Trains a logistic regression model on the training data
- 📊 Evaluates the model using `classification_report`, showing precision, recall, and F1-score per class
- 📁 Returns a dictionary of useful components: the model, data splits, label encoder, and feature names

**✅ How to Use the Function**

You can apply this function to any dataset with:
- A categorical target column (e.g. tour product names)
- A set of binary indicator features (0 or 1)
- An optional ID column used to uniquely identify rows

To adjust model behavior:
- Use parameters like `penalty`, `C`, and `solver` to apply **Lasso (L1)**, **Ridge (L2)**, or **ElasticNet** regularization
- Use `class_weight='balanced'` to automatically adjust for **class imbalance** in your multi-class classification tasks — this helps ensure that minority classes are not ignored during training
- Modify `test_size` or `random_state` to control how your validation set is split
- Specify `drop_cols` to exclude any additional metadata columns from modeling

This gives us a solid foundation to experiment with different models, tune hyperparameters, and eventually deploy our trained classifier into a real-world AI workflow.


In [ ]:
def train_multiclass_logistic_regression_model(
    df,
    target_col='product_name',
    id_col='user_id',
    drop_cols=None,
    test_size=0.2,
    random_state=42,
    class_weight='balanced',
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000
):
    """
    Train and evaluate a logistic regression model on the Explore California product recommendation dataset.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing user attributes and a multi-class target label.

    target_col : str, default='product_name'
        The name of the multi-class target column.

    id_col : str, default='user_id'
        The name of the unique identifier column (excluded from training features).

    drop_cols : list of str, optional
        Additional columns to exclude from training (e.g., ['product_name', 'user_id']).
        If None, will default to [target_col, id_col].

    test_size : float, default=0.2
        Proportion of the dataset to include in the validation split.

    random_state : int, default=42
        Controls shuffling for reproducibility in the train/test split.

    class_weight : str, dict, or None, default='balanced'
        Weights associated with classes. Useful when classes are imbalanced.
        - Use `'balanced'` to automatically adjust weights inversely proportional to class frequencies.
        - You can also provide a custom dictionary of weights, or use `None` for no weighting.

    penalty : str, default='l2'
        Type of regularization to apply:
        - 'l2' for Ridge
        - 'l1' for Lasso
        - 'elasticnet' for a mix (requires solver='saga')

    C : float, default=1.0
        Inverse of regularization strength. Smaller values imply stronger regularization.

    solver : str, default='lbfgs'
        Algorithm to use in the optimization problem.
        - 'liblinear' supports L1
        - 'saga' supports elasticnet

    max_iter : int, default=1000
        Maximum number of iterations for solver to converge.

    Returns
    -------
    dict
        Dictionary containing the trained model, label encoder, and training/validation splits.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report

    # ---------------------------------------
    # Select features and target
    # ---------------------------------------
    if drop_cols is None:
        drop_cols = [target_col, id_col]

    X = df.drop(columns=drop_cols)
    y = df[target_col]

    # 🔠 Encode the target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # 🔀 Create train/validation split
    # You can adjust test_size or stratify behavior here
    X_train, X_val, y_train, y_val = train_test_split(
        X,
        y_encoded,
        test_size=test_size,
        stratify=y_encoded,
        random_state=random_state
    )

    # 📦 Train logistic regression model
    # You can modify penalty, C, solver, or class_weight to control behavior:
    # - class_weight='balanced' is recommended for class imbalance
    # - penalty='l2' → Ridge
    # - penalty='l1' → Lasso (requires solver='liblinear')
    # - penalty='elasticnet' → L1 + L2 mix (requires solver='saga')
    model = LogisticRegression(
        class_weight=class_weight,
        penalty=penalty,
        C=C,
        solver=solver,
        max_iter=max_iter,
        random_state=random_state
    )
    model.fit(X_train, y_train)

    # 📊 Evaluate the model using precision, recall, and F1 score
    y_pred = model.predict(X_val)
    print(classification_report(y_val, y_pred, target_names=label_encoder.classes_))

    # Return all components for downstream use or export
    return {
        "model": model,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist()
    }


In [ ]:
# This should take ~30 seconds to run!
product_recommendations_logistic_regression = train_multiclass_logistic_regression_model(product_recommendation_df)

In [ ]:

# Let's also demonstrate how to save the model artefacts using joblib
# This is useful for later deployment or inference!
import joblib

# Choose only certain components parts of the model output
pickle_outputs = ["model", "label_encoder", "attribute_names"]
filtered_outputs = {key: product_recommendations_logistic_regression[key] for key in pickle_outputs}

# Write it out as a pickle Python dictionary object
joblib.dump(filtered_outputs, "logistic_model_outputs.pkl")

# This is how we can load it back into our Notebook scope!
model_artefacts = joblib.load("logistic_model_outputs.pkl")

# Refer to each element within the object
ml_model = model_artefacts["model"]
ml_label_encoder = model_artefacts["label_encoder"]
ml_attribute_names = model_artefacts["attribute_names"]

## 2.3 Model Performance Summary

Our logistic regression model achieved **overall accuracy of 97%** across 10 product classes in the validation set.

Key takeaways:

- ✅ **High-performing classes** include:
  - *California Classics*, *Southern Skies Trail*, and *Sun & Stone Discovery* — all with precision and recall above 95%.
  
- ⚠️ **Lower-performing but small classes**:
  - *Epic NorCal Expedition*, *Giant Trees & Granite Dreams*, and *Golden State Wonders* had **lower precision** (~70–75%) but **very high recall** (98–99%), meaning the model tends to over-predict these classes slightly but still catches most true cases.

- 📐 **Macro average F1-score** is **0.93**, indicating solid average performance across all classes.
- ⚖️ **Weighted average F1-score** is **0.97**, showing strong results especially for more frequent classes.

This strong baseline provides a great starting point for experimenting with other models and tuning techniques.


## 2.4 Visualizing Model Performance with a Confusion Matrix

Once we’ve trained our multi-class logistic regression model, it’s important to go beyond just accuracy scores and look at **where** our model is getting predictions right — and where it’s going wrong.

A **confusion matrix** gives us a detailed breakdown of how often each class (in our case, tour product) is correctly predicted vs. confused with others. This can reveal:

- Which classes the model handles confidently and accurately ✅  
- Which classes tend to get confused with others 🤔  
- Whether certain products are underrepresented or difficult to classify ⚠️  

For example, if two tours are similar in theme or user preferences, the model might struggle to differentiate between them — and this would show up as off-diagonal values in the matrix.

We’ll implement a reusable `plot_confusion_matrix()` function to help us visualize this breakdown.  
It takes in the model output dictionary and displays a color-coded matrix with prediction vs. actual classes.

> This step helps guide future improvements — such as adjusting class weights, rebalancing data, or engineering new features.


In [ ]:
def plot_confusion_matrix(model_output, title="Confusion Matrix"):
    """
    Plot a labeled confusion matrix with both raw counts and row-normalized percentages.

    Parameters
    ----------
    model_output : dict
        Dictionary returned from a model training function (e.g., `train_multiclass_logistic_regression_model`)
        that must include:
        - 'model': Trained scikit-learn classifier
        - 'X_val': Validation feature matrix
        - 'y_val': Encoded ground truth labels for validation set
        - 'label_encoder': Fitted LabelEncoder used to transform the target

    title : str, default="Confusion Matrix – Multiclass Classification"
        Custom title to show at the top of the plot

    This function will:
    -------------------
    - Predict outcomes on validation data using the trained model
    - Compute the confusion matrix (true vs. predicted labels)
    - Normalize the matrix row-wise to calculate per-class prediction proportions
    - Plot a heatmap showing both raw counts and normalized percentages in each cell
    - Label axes with the decoded class names for easy interpretation

    Notes
    -----
    - Use this plot to visually assess where your model is performing well or making systematic errors.
    - Works best for multi-class classification tasks, especially when paired with imbalanced data.
    - This version includes both absolute values and percentages in each cell to aid interpretation.
    """

    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import numpy as np

    # 🔍 Extract components from model output
    model = model_output["model"]
    X_val = model_output["X_val"]
    y_val = model_output["y_val"]
    label_encoder = model_output["label_encoder"]
    class_names = label_encoder.classes_

    # 🔮 Generate predictions
    y_pred = model.predict(X_val)

    # 📊 Compute the confusion matrix
    cm = confusion_matrix(y_val, y_pred)

    # Normalize the confusion matrix by row (i.e., by the total number of true labels per class)
    # This allows us to calculate what percentage of actual class 'i' was predicted as class 'j'
    # The result is a matrix of proportions, where each row sums to 1 (or 100%)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    # 🎨 Plot heatmap
    fig, ax = plt.subplots(figsize=(10, 10))
    # cividis is useful for colorblind-friendly visualizations
    # You can change the colormap to 'Blues', 'Greens' or other options
    im = ax.imshow(cm_normalized, interpolation='nearest')
    ax.figure.colorbar(im, ax=ax)

    # 🏷️ Axis labels and ticks
    ax.set_xticks(np.arange(len(class_names)))
    ax.set_yticks(np.arange(len(class_names)))
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    ax.set_xlabel("Predicted Label")
    ax.set_ylabel("True Label")
    ax.set_title(title)
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

    # 🔢 Annotations (aligned to cell centers)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            count = cm[i, j]
            percent = cm_normalized[i, j] * 100
            text = f"{count}\n{percent:.1f}%"
            ax.text(j, i, text, ha='center', va='center', color='black', fontsize=10)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_confusion_matrix(
  product_recommendations_logistic_regression,
  "Logistic Regression - Product Recommendations"
)

## 2.3 Random Forest with Cross-Validation and Hyperparameter Search

In this step, we’ll train a **Random Forest classifier** using **Stratified K-Fold cross-validation** and **randomized hyperparameter search** to improve predictive performance on our Explore California product recommendation dataset.

### What this function implements:

- ✅ **Train/validation split** using `train_test_split` with `stratify` to ensure class proportions are preserved
- 🔠 **Label encoding** of the target column (`product_name`) using `LabelEncoder`
- ⚙️ **Random Forest classifier** inside a `Pipeline`, with support for class weighting (`class_weight='balanced'`) to handle class imbalance
- 🔁 **Stratified K-Fold Cross-Validation** to get a more reliable estimate of model performance:
  - The data is split into `k` folds (typically 5 but we'll reduce our default to 3 so it will run easier on our Google Colab environment)
  - In each iteration, one fold is used as the validation set and the remaining `k-1` folds are used for training
  - **Stratification** ensures that the class distribution remains similar across each fold
  - This helps prevent biased evaluation, especially when dealing with imbalanced classes
- 🔍 **Randomized Hyperparameter Search** via `RandomizedSearchCV`:
  - Instead of exhaustively searching all combinations like Grid Search, it **randomly samples** a fixed number (`n_iter`) of combinations from a specified parameter distribution
  - This makes it much faster while still exploring a broad set of configurations
  - Hyperparameters being tuned include:
    - `n_estimators` (number of trees)
    - `max_depth` (maximum depth of trees)
    - `min_samples_split` (min samples required to split an internal node)
    - `min_samples_leaf` (min samples required to be a leaf node)

---

### Customization and control:

You can pass your own values to control the training process:

- `test_size`: Fraction of data to set aside for final validation
- `n_splits`: Number of folds for cross-validation
- `n_iter`: Number of random combinations to try in hyperparameter search
- `scoring`: Evaluation metric used during tuning (e.g. `'f1_weighted'`)
- `class_weight`: Strategy for handling class imbalance (`'balanced'` is a common default)
- `param_dist`: A dictionary defining the range or distribution of hyperparameters to sample from

This approach gives you a high quality, tunable modeling function — ideal for building robust classification models.


In [ ]:
def train_kfold_random_forest_model(
    df,
    target_col='product_name',
    id_col='user_id',
    drop_cols=None,
    test_size=0.2,
    random_state=42,
    scoring='f1_weighted',
    n_iter=5,
    n_splits=3,
    class_weight='balanced',
    param_dist=None
):
    """
    Train and evaluate a Random Forest model using Stratified K-Fold cross-validation and randomized search.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing features and a multi-class classification target.

    target_col : str, default='product_name'
        The name of the multi-class target column.

    id_col : str, default='user_id'
        The name of the unique identifier column (excluded from training features).

    drop_cols : list of str, optional
        Additional columns to exclude from training. If None, defaults to [target_col, id_col].

    test_size : float, default=0.2
        Proportion of the dataset to use as the validation split.

    random_state : int, default=42
        Seed used for reproducibility across train/test splits and random search.

    scoring : str, default='f1_weighted'
        Scoring metric to optimize during cross-validation (e.g., 'accuracy', 'precision_weighted').

    n_iter : int, default=5
        Number of parameter combinations to try in `RandomizedSearchCV`.

    n_splits : int, default=3
        Number of cross-validation folds for Stratified K-Fold.

    class_weight : str, dict or None, default='balanced'
        Class weight setting for the Random Forest.
        - Use 'balanced' to correct for class imbalance.

    param_dist : dict, optional
        Dictionary specifying the hyperparameter search space.
        If None, a default range will be used.

    Returns
    -------
    dict
        Dictionary containing the best estimator, label encoder, and train/validation splits.
    """

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report
    from scipy.stats import randint

    # ---------------------------------------
    # Feature and target selection
    # ---------------------------------------
    if drop_cols is None:
        drop_cols = [target_col, id_col]

    X = df.drop(columns=drop_cols)
    y = df[target_col]

    # 🔠 Encode the target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # 🔀 Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_encoded, test_size=test_size, stratify=y_encoded, random_state=random_state
    )

    # 📦 Build pipeline with Random Forest
    pipeline = Pipeline([
        ("clf", RandomForestClassifier(class_weight=class_weight, random_state=random_state))
    ])

    # 🔧 Hyperparameter space
    if param_dist is None:
        param_dist = {
            "clf__n_estimators": randint(50, 300),
            "clf__max_depth": randint(5, 30),
            "clf__min_samples_split": randint(2, 10),
            "clf__min_samples_leaf": randint(1, 10)
        }

    # 🔁 Cross-validation strategy
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # 🔍 Randomized hyperparameter search
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=kfold,
        scoring=scoring,
        verbose=1,
        n_jobs=-1,
        random_state=random_state
    )

    # 🚂 Train the model
    random_search.fit(X_train, y_train)

    # 🏆 Report best results
    print("Best F1 Weighted Score:", random_search.best_score_)
    print("Best Parameters:", random_search.best_params_)

    # 📊 Evaluate on validation data
    y_val_pred = random_search.predict(X_val)
    print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

    # 🔁 Return for downstream use
    return {
        "model": random_search.best_estimator_,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist()
    }


In [ ]:
# This should take ~1 minute to run!
product_recommendations_random_forest = train_kfold_random_forest_model(product_recommendation_df)

In [ ]:
plot_confusion_matrix(
  product_recommendations_random_forest,
  "Random Forest - Product Recommendations"
)

## 2.4 Model Performance Comparison: Random Forest vs Logistic Regression

We trained two different models to predict user tour preferences:

- A **Random Forest Classifier** with randomized hyperparameter tuning
- A **Multiclass Logistic Regression** model with balanced class weights

### ✅ Summary of Results

| Metric               | Random Forest | Logistic Regression |
|----------------------|---------------|----------------------|
| **Accuracy**         | 0.58          | 0.97                 |
| **Macro F1-score**   | 0.52          | 0.93                 |
| **Weighted F1-score**| 0.58          | 0.97                 |

### 📊 Key Observations

- The **logistic regression model significantly outperforms** the random forest across all metrics, especially on:
  - **Macro F1**: which treats all classes equally — strong evidence that logistic regression generalizes better to minority classes.
  - **Accuracy** and **Weighted F1**: which factor in class frequencies — logistic regression still wins comfortably.

- **Class-by-class performance**:
  - Logistic regression shows **consistently high precision and recall** across all tour products.
  - Random forest shows **weaker precision/recall**, particularly on low-support classes (e.g., Epic NorCal, Granite Dreams).

### 🤔 Why might a simpler logistic regression model perform better?

1. **Feature space is binary and sparse**: Logistic regression often works very well with binary inputs and linear decision boundaries. Decision trees can overfit this kind of data, especially when sample sizes are small for some classes.

2. **Class imbalance is handled explicitly** in logistic regression via `class_weight='balanced'`, while random forest may require more careful tuning to avoid bias toward frequent classes.

3. **Regularization** in logistic regression (L2 penalty) helps prevent overfitting and improves generalization — especially useful in high-dimensional spaces with many correlated features.

4. **Random forest may be under-tuned**: Even with randomized search, it’s possible that the model didn’t find optimal hyperparameters for this task or needed more iterations/folds to stabilize.

5. **Overfitting small classes**: Random forest can create overly complex trees that memorize training examples from smaller classes, which harms generalization.

---

📌 **Conclusion**: In this case, the **simpler logistic regression model performs best** — reinforcing the idea that more complex models don’t always lead to better results, especially when working with well-structured binary features and limited data per class.


## 2.5 Early Stopping with LightGBM

In this section, we introduce a powerful training technique called **early stopping**, which helps prevent overfitting and speeds up training for gradient boosting models like LightGBM.

### 🧠 What is Early Stopping?

Early stopping is a **regularization technique** used during model training that monitors performance on a validation set. Training halts when the model's performance stops improving after a certain number of rounds — preventing it from overfitting to the training data.

- In our case, we monitor the **multi-class log loss** on the validation set.
- If the model doesn't improve for `20 rounds`, training stops early and the best iteration is used for prediction.

### ⚙️ Adapting Pipelines for LightGBM

Unlike scikit-learn’s standard classifiers, **LightGBM has its own training interface** that supports early stopping via callbacks. This requires a few changes:

- We **don’t use a `Pipeline`** from scikit-learn — instead, we call `.fit()` directly on the LightGBM model.
- We pass the `eval_set`, `eval_metric`, and `early_stopping` **as arguments** to the `fit()` method.
- The `LGBMClassifier` still accepts scikit-learn-compatible parameters like `class_weight`, `n_estimators`, and `learning_rate`.

### 🔍 Key Parameters Used

- `n_estimators=10000`: A large number of boosting rounds — early stopping will decide the actual stopping point.
- `early_stopping(stopping_rounds=20)`: Stops training if no improvement after 20 rounds.
- `class_weight='balanced'`: Helps adjust for class imbalance.
- `eval_metric='multi_logloss'`: Appropriate for multi-class classification.

---

📌 **Why use early stopping?**  
It’s especially useful for boosting models like LightGBM and XGBoost, where too many iterations can lead to overfitting. Early stopping helps us **automatically find the optimal number of boosting rounds** — improving both accuracy and efficiency.


In [ ]:
def train_lightgbm_early_stopping_model(
    df,
    target_col='product_name',
    id_col='user_id',
    drop_cols=None,
    test_size=0.2,
    random_state=42,
    learning_rate=0.1,
    n_estimators=10000,
    class_weight='balanced',
    early_stopping_rounds=20,
    eval_metric='multi_logloss'
):
    """
    Train and evaluate a LightGBM model with early stopping for multi-class classification.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing user attributes and a multi-class target label.

    target_col : str, default='product_name'
        The name of the multi-class target column.

    id_col : str, default='user_id'
        The name of the identifier column to exclude from training.

    drop_cols : list of str, optional
        Additional columns to exclude from training. If None, defaults to [target_col, id_col].

    test_size : float, default=0.2
        Proportion of the dataset to include in the validation split.

    random_state : int, default=42
        Random seed for reproducibility in train/test splitting and model training.

    learning_rate : float, default=0.1
        Learning rate for boosting. Smaller values may yield better generalization but require more boosting rounds.

    n_estimators : int, default=10000
        Maximum number of boosting rounds. Training may stop earlier if early stopping criteria are met.

    class_weight : str, dict, or None, default='balanced'
        Weights associated with classes. 'balanced' automatically adjusts for class imbalance.

    early_stopping_rounds : int, default=20
        Number of rounds to wait for improvement before stopping training early.

    eval_metric : str, default='multi_logloss'
        Evaluation metric used for early stopping.

    Returns
    -------
    dict
        Dictionary containing the trained model, label encoder, and training/validation splits.
    """

    from lightgbm import LGBMClassifier, early_stopping
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report

    # ---------------------------------------
    # Feature and target selection
    # ---------------------------------------
    if drop_cols is None:
        drop_cols = [target_col, id_col]

    X = df.drop(columns=drop_cols)
    y = df[target_col]

    # 🔠 Encode target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # 🔀 Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_encoded, test_size=test_size, stratify=y_encoded, random_state=random_state
    )

    # 📦 Configure LightGBM classifier
    model = LGBMClassifier(
        objective='multiclass',
        num_class=len(label_encoder.classes_),
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        class_weight=class_weight,
        random_state=random_state
    )

    # ⚡ Train with early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=eval_metric,
        callbacks=[early_stopping(stopping_rounds=early_stopping_rounds)]
    )

    # 📊 Evaluate on validation set
    y_val_pred = model.predict(X_val)
    print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

    # 🔁 Return model and metadata for downstream use
    return {
        "model": model,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist()
    }


In [ ]:
# This should take ~1 minute to run!
product_recommendations_lightgbm = train_lightgbm_early_stopping_model(product_recommendation_df)


In [ ]:
plot_confusion_matrix(product_recommendations_lightgbm, "LightGBM - Product Recommendations")

## 2.5 LightGBM with Optuna (Bayesian Optimization)

In this section, we’ll train a **LightGBM model** using **Optuna**, a Python library for **Bayesian hyperparameter optimization**. Compared to traditional grid or random search, this approach is more efficient and intelligent — especially when tuning many hyperparameters.

---

### 🧠 What is Bayesian Optimization?

Think of hyperparameter tuning like searching for the best restaurant in a new city:

- **Grid search** is like trying every single restaurant on a giant map — time-consuming and inefficient.
- **Random search** is like picking restaurants at random and hoping one is great.
- **Bayesian optimization** is like asking locals for recommendations, trying the best one, then adjusting your search based on what you liked — getting smarter with each try.

Optuna implements this smarter strategy by learning from past trials and focusing on the most promising combinations.

---

### 🧪 What does this function do?

1. **Defines an objective function**  
   - We wrap the model training and evaluation into a function that returns a **macro-averaged F1 score**, which is a good fit for multi-class tasks.
   - ✅ Inside this objective function, we use **early stopping** during model training to avoid overfitting and reduce unnecessary computation.

2. **Launches an Optuna study**  
   - The study runs 10 trials, each one suggesting a different set of LightGBM hyperparameters (like `learning_rate`, `num_leaves`, `lambda_l1`, etc.).
   - After each trial, Optuna updates its internal model to focus on better-performing regions of the hyperparameter space.

3. **Trains the final model**  
   - After optimization, we retrain a LightGBM model using the best hyperparameters discovered by Optuna.

4. **Evaluates model performance**  
   - We report the **accuracy**, **macro F1 score**, and a **full classification report** on the validation set.

---

### ⚙️ Why this matters

- **Faster convergence**: Bayesian optimization often finds strong models with fewer trials.
- **Efficient training**: By using **early stopping in each trial**, we ensure that every model is trained as well as possible — without wasting time on poor configurations.
- **Better performance**: We're directly optimizing for the metric that matters most.
- **Powerful for complex models**: This approach shines when tuning multiple parameters in high-capacity models like LightGBM.

If you've tried grid or random search, this method is the natural next step — combining **smart exploration** with **practical safeguards like early stopping** for faster, better results.


In [ ]:
def train_lightgbm_optuna_model(
    df,
    target_col='product_name',
    id_col='user_id',
    drop_cols=None,
    test_size=0.2,
    random_state=42,
    n_trials=10,
    scoring_metric='macro'
):
    """
    Train and evaluate a LightGBM model using Optuna for Bayesian hyperparameter optimization.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing user features and a multi-class classification target.

    target_col : str, default='product_name'
        The name of the multi-class target column.

    id_col : str, default='user_id'
        The name of the identifier column to exclude from training.

    drop_cols : list of str, optional
        Additional columns to exclude from training. If None, defaults to [target_col, id_col].

    test_size : float, default=0.2
        Proportion of the dataset to include in the validation split.

    random_state : int, default=42
        Random seed for reproducibility in data splitting and model training.

    n_trials : int, default=10
        Number of trials for Optuna to explore in the hyperparameter search space.

    scoring_metric : str, default='macro'
        Type of F1 averaging to use when evaluating performance ('macro', 'weighted', etc.).

    Returns
    -------
    dict
        Dictionary containing the best trained model, label encoder, and training/validation data.
    """

    from lightgbm import LGBMClassifier, early_stopping
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import classification_report, f1_score, accuracy_score
    import optuna

    # ---------------------------------------
    # Feature and target selection
    # ---------------------------------------
    if drop_cols is None:
        drop_cols = [target_col, id_col]

    X = df.drop(columns=drop_cols)
    y = df[target_col]

    # 🔠 Encode the target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # 🔀 Split into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_encoded, test_size=test_size, stratify=y_encoded, random_state=random_state
    )

    # ---------------------------------------
    # Define Optuna objective function
    # ---------------------------------------
    def objective(trial):
        params = {
            "objective": "multiclass",
            "num_class": len(label_encoder.classes_),
            "metric": "multi_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "random_state": random_state,
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 15, 100),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        }

        model = LGBMClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="multi_logloss",
            callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
        )

        preds = model.predict(X_val)
        return f1_score(y_val, preds, average=scoring_metric)

    # 🔍 Run Optuna study
    sampler = optuna.samplers.TPESampler(seed=10)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=n_trials)

    # 🏆 Retrieve best parameters
    best_params = study.best_trial.params
    best_params["objective"] = "multiclass"
    best_params["num_class"] = len(label_encoder.classes_)

    # 🚂 Train final model with best parameters
    final_model = LGBMClassifier(**best_params)
    final_model.fit(X_train, y_train)

    # 📊 Evaluate final model
    y_val_pred = final_model.predict(X_val)
    print("Accuracy:", accuracy_score(y_val, y_val_pred))
    print(f"F1 Score ({scoring_metric}):", f1_score(y_val, y_val_pred, average=scoring_metric))
    print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

    # 🔁 Return model and training artifacts
    return {
        "model": final_model,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist()
    }


In [ ]:
# This should take ~2 minutes to run!
product_recommendations_bayesian_optimization_lightgbm = train_lightgbm_optuna_model(product_recommendation_df)


In [ ]:
plot_confusion_matrix(
  product_recommendations_bayesian_optimization_lightgbm,
  "Optuna LightGBM - Product Recommendations"
)

## 2.6 Final Model Comparison

We’ve trained and evaluated four models for our product recommendation task:

1. **Logistic Regression**
2. **Random Forest**
3. **LightGBM with Early Stopping**
4. **LightGBM with Optuna Bayesian Optimization**

Before we dive into detailed insights, here’s a high-level summary of the performance metrics:

| Model                           | Accuracy | Macro F1 Score | Weighted F1 Score |
|--------------------------------|----------|----------------|-------------------|
| **Logistic Regression**        | 0.97     | 0.93           | 0.97              |
| **Random Forest**              | 0.59     | 0.52           | 0.59              |
| **LightGBM (Early Stopping)**  | 0.80     | 0.64           | 0.79              |
| **LightGBM (Optuna Optimized)**| 0.70     | 0.56           | 0.69              |

---

### ✅ 1. Logistic Regression

- **Accuracy**: 97%  
- **Macro F1 Score**: 0.93  
- **Weighted F1 Score**: 0.97  

**Observations:**
- Strong precision and recall across all classes.
- Consistently high performance even for minority classes (e.g., *Epic NorCal Expedition*, *Giant Trees*).
- Nearly perfect classification for popular tours.

✔️ **Best overall performance.**  
📌 *Simple, robust, and surprisingly well-suited to the dataset.*

---

### 🌲 2. Random Forest

- **Accuracy**: 59%  
- **Macro F1 Score**: 0.52  
- **Weighted F1 Score**: 0.59  

**Observations:**
- Struggles with precision and recall across almost all classes.
- High variance — some classes overfit, others underperform.
- Appears unable to capture complex feature interactions meaningfully in this setting.

⚠️ **Worst overall performance.**  
📌 *Could indicate overfitting or weak signal extraction from binary features.*

---

### 🌟 3. LightGBM with Early Stopping

- **Accuracy**: 80%  
- **Macro F1 Score**: 0.64  
- **Weighted F1 Score**: 0.79  

**Observations:**
- Stronger performance than Random Forest.
- Very high precision but **poor recall** for minority classes (e.g., *Epic NorCal*, *Golden State Wonders*).
- May be over-prioritizing dominant classes.

🟡 **Moderate performance, with high class imbalance sensitivity.**  
📌 *Early stopping helps prevent overfitting but may not be enough without class-specific tuning.*

---

### 🔧 4. LightGBM + Optuna (Bayesian Optimization)

- **Accuracy**: 70%  
- **Macro F1 Score**: 0.56  
- **Weighted F1 Score**: 0.69  

**Observations:**
- Better recall for minority classes compared to early stopping model.
- More balanced precision–recall tradeoff.
- Still trails behind Logistic Regression.
- Might still be slightly underfit and could do with more than 10 study trials.

🟠 **Improvement via Bayesian tuning — but still not best in class.**  
📌 *Shows value of tuning, but LightGBM still seems less well-suited to this dataset.*

---

### 🤔 Why Did **Logistic Regression** Win?

Even though it’s a linear model, Logistic Regression performed **significantly better** than the more complex models. Here’s why:

| Reason | Explanation |
|--------|-------------|
| ✅ **Binary Inputs** | The dataset uses **binary features**. Logistic regression handles these well without needing complex tree splits. |
| ✅ **Low Feature Interaction** | If interactions between features are limited, linear models may generalize better than over-parameterized tree models. |
| ✅ **Well-Calibrated Class Balance** | Logistic regression with stratified train/test splits can handle class imbalance reasonably with fewer tuning knobs. |
| ⚠️ **Tree Models Overfit** | Random Forest and LightGBM can overfit if the signal is weak or if hyperparameters aren't tuned precisely — especially with small class sizes. |
| ✅ **Class Separation is Linearly Separable** | The features likely provide **enough separation** in a linear decision space — no complex boundaries needed. |

---

### 🔚 Final Thoughts

- **Start simple.** Always benchmark with logistic regression — you might be surprised.
- **Don’t blindly trust complexity.** Tree-based models offer flexibility, but may **underperform** if the data structure doesn’t demand it.
- **Tune with care.** Even Optuna can’t save a model that doesn’t fit the problem’s shape.

Let’s now take these insights and think about how we could further improve our tree-based models:
- Try **feature engineering** or interaction terms.
- Use **calibrated class weights** or focal loss for imbalanced classes.
- Explore **embedding** sparse features if the space becomes large.


# 3. Binary Classification

In this section, we’ll shift our focus from multi-class product recommendations to a **binary classification task** — predicting whether a user will purchase *any* product within a defined time period. In our case, it's within 30 days from the provided `label_date`.

We'll be using our binary classification dataset `purchase_prediction_df`, which includes both user attributes and pre-assigned training, validation, and test periods.

Here's what we'll be covering in this section:

1. **Temporal Data Splitting**  
   We'll use the provided `train`, `val`, and `test` flags in the dataset — simulating how we typically train on past data and validate on future outcomes.

2. **Robust Data Preprocessing**  
   We'll adapt our `scikit-learn` pipeline to:
   - Handle missing values using a simple imputation strategy  
   - Prepare binary target labels for classification

3. **Model 1: Logistic Regression**  
   We’ll fit a simple, interpretable baseline model and evaluate how well it separates buyers from non-buyers.

4. **Model 2: LightGBM with Optuna Optimization**  
   Using **Bayesian optimization**, we’ll tune a more powerful model to see if we can improve upon the baseline.

5. **Model Evaluation**  
   Instead of accuracy, we’ll use:
   - **AUC (Area Under the ROC Curve)**  
   - **ROC Curves** to visualize true positive vs. false positive tradeoffs

6. **Model Explainability with SHAP and LIME**  
   Predictive performance is important — but equally critical is being able to understand and explain **why** your model made a certain decision.

   In this final step, we'll:
   - Use **SHAP (SHapley Additive exPlanations)** to get global and local feature importance scores based on game theory.
   - Use **LIME (Local Interpretable Model-Agnostic Explanations)** to explain individual predictions by fitting simple models locally.

   These techniques help us:
   - Build trust with stakeholders  
   - Detect spurious correlations  
   - Comply with model transparency requirements in production

## 3.1 Logistic Regression Model

In [ ]:
def train_binary_logistic_regression_model_with_temporal_split(
    df,
    target_col='label',
    positive_class=1,
    id_col='user_id',
    drop_cols=['label_date'],
    period_col='period',
    class_weight='balanced',
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    max_iter=1000,
    random_state=42
):
    """
    Train and evaluate a binary logistic regression model using temporal splits.
    Plots ROC and Precision-Recall curves for train, validation, and test sets (if available).

    Returns
    -------
    dict
        Trained model, predictions, probabilities, label encoder, and evaluation data splits.
    """
    import matplotlib.pyplot as plt
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        classification_report, roc_auc_score, roc_curve,
        precision_recall_curve
    )
    from sklearn.preprocessing import LabelEncoder

    # ---------------------------------------
    # Preprocessing
    # ---------------------------------------
    df = df.fillna(0)
    drop_cols = drop_cols or []
    drop_cols = list(set(drop_cols + [target_col, id_col, period_col]))

    X = df.drop(columns=drop_cols)
    y_raw = df[target_col]

    if y_raw.nunique() > 2:
        raise ValueError("Target column must be binary.")

    if y_raw.dtype == 'object' or not set(y_raw.unique()).issubset({0, 1}):
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y_raw == positive_class)
    else:
        label_encoder = None
        y_encoded = (y_raw == positive_class).astype(int)

    # ---------------------------------------
    # Temporal Split
    # ---------------------------------------
    period = df[period_col]
    train_mask = period == "train"
    val_mask = period == "validation"
    test_mask = period == "test"

    X_train, y_train = X[train_mask], y_encoded[train_mask]
    X_val, y_val = X[val_mask], y_encoded[val_mask]
    X_test, y_test = X[test_mask], y_encoded[test_mask] if test_mask.any() else (None, None)

    # ---------------------------------------
    # Train Model
    # ---------------------------------------
    model = LogisticRegression(
        class_weight=class_weight,
        penalty=penalty,
        C=C,
        solver=solver,
        max_iter=max_iter,
        random_state=random_state
    )
    model.fit(X_train, y_train)

    # ---------------------------------------
    # Evaluate on Train Set
    # ---------------------------------------
    y_train_pred = model.predict(X_train)
    y_train_proba = model.predict_proba(X_train)[:, 1]
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
    precision_train, recall_train, _ = precision_recall_curve(y_train, y_train_proba)
    auc_train = roc_auc_score(y_train, y_train_proba)

    print("📊 Train Classification Report:")
    print(classification_report(y_train, y_train_pred))

    # ---------------------------------------
    # Evaluate on Validation Set
    # ---------------------------------------
    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]
    fpr_val, tpr_val, _ = roc_curve(y_val, y_val_proba)
    precision_val, recall_val, _ = precision_recall_curve(y_val, y_val_proba)
    auc_val = roc_auc_score(y_val, y_val_proba)

    print("\n📊 Validation Classification Report:")
    print(classification_report(y_val, y_val_pred))

    # ---------------------------------------
    # Evaluate on Test Set (Optional)
    # ---------------------------------------
    if test_mask.any():
        y_test_pred = model.predict(X_test)
        y_test_proba = model.predict_proba(X_test)[:, 1]
        fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)
        precision_test, recall_test, _ = precision_recall_curve(y_test, y_test_proba)
        auc_test = roc_auc_score(y_test, y_test_proba)

        print("\n📊 Test Classification Report:")
        print(classification_report(y_test, y_test_pred))
    else:
        y_test_proba, fpr_test, tpr_test = None, None, None
        precision_test, recall_test, auc_test = None, None, None

    # ---------------------------------------
    # Plot ROC + Precision-Recall Curves
    # ---------------------------------------
    fig, axes = plt.subplots(2, 3, figsize=(18, 8))

    # ROC Curves
    axes[0, 0].plot(fpr_train, tpr_train, label=f"AUC = {auc_train:.2f}", lw=2)
    axes[0, 0].plot([0, 1], [0, 1], linestyle="--", color="gray")
    axes[0, 0].set_title("ROC Curve (Train)")
    axes[0, 0].set_xlabel("False Positive Rate")
    axes[0, 0].set_ylabel("True Positive Rate")
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(fpr_val, tpr_val, label=f"AUC = {auc_val:.2f}", lw=2)
    axes[0, 1].plot([0, 1], [0, 1], linestyle="--", color="gray")
    axes[0, 1].set_title("ROC Curve (Validation)")
    axes[0, 1].set_xlabel("False Positive Rate")
    axes[0, 1].set_ylabel("True Positive Rate")
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    if fpr_test is not None:
        axes[0, 2].plot(fpr_test, tpr_test, label=f"AUC = {auc_test:.2f}", lw=2)
        axes[0, 2].plot([0, 1], [0, 1], linestyle="--", color="gray")
        axes[0, 2].set_title("ROC Curve (Test)")
        axes[0, 2].set_xlabel("False Positive Rate")
        axes[0, 2].set_ylabel("True Positive Rate")
        axes[0, 2].legend()
        axes[0, 2].grid(True)

    # Precision-Recall Curves
    axes[1, 0].plot(recall_train, precision_train, label="Train", lw=2)
    axes[1, 0].set_title("Precision-Recall (Train)")
    axes[1, 0].set_xlabel("Recall")
    axes[1, 0].set_ylabel("Precision")
    axes[1, 0].grid(True)

    axes[1, 1].plot(recall_val, precision_val, label="Validation", lw=2)
    axes[1, 1].set_title("Precision-Recall (Validation)")
    axes[1, 1].set_xlabel("Recall")
    axes[1, 1].set_ylabel("Precision")
    axes[1, 1].grid(True)

    if precision_test is not None:
        axes[1, 2].plot(recall_test, precision_test, label="Test", lw=2)
        axes[1, 2].set_title("Precision-Recall (Test)")
        axes[1, 2].set_xlabel("Recall")
        axes[1, 2].set_ylabel("Precision")
        axes[1, 2].grid(True)

    plt.tight_layout()
    plt.show()

    # ---------------------------------------
    # Return Training Artifacts
    # ---------------------------------------
    return {
        "model": model,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "X_test": X_test if test_mask.any() else None,
        "y_test": y_test if test_mask.any() else None,
        "y_proba_train": y_train_proba,
        "y_proba_val": y_val_proba,
        "y_proba_test": y_test_proba if test_mask.any() else None,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist(),
        "curve_plot": fig
    }


In [ ]:
# this should take 30 seconds to run!
purchase_prediction_logistic_regression = train_binary_logistic_regression_model_with_temporal_split(
  purchase_prediction_df
)

## 3.2 LightGBM Model

In [ ]:
def train_lightgbm_optuna_binary_with_temporal_split(
    df,
    target_col='label',
    positive_class=1,
    id_col='user_id',
    drop_cols=['label_date'],
    period_col='period',
    n_trials=20,
    scoring_metric='binary',
    random_state=42
):
    """
    Train a binary LightGBM model using Optuna for hyperparameter tuning with temporal train/val/test splits.
    Includes train, validation, and test ROC and Precision-Recall curves.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame with features, binary target, and 'period' column.

    target_col : str, default='label'
        Binary classification target column.

    positive_class : int or str, default=1
        Value treated as positive class.

    id_col : str, default='user_id'
        Identifier column to exclude from training.

    drop_cols : list of str, optional
        Additional columns to exclude (e.g., ['label_date']).

    period_col : str, default='period'
        Column containing 'train', 'validation', and 'test' labels.

    n_trials : int, default=20
        Number of Optuna trials.

    scoring_metric : str, default='binary'
        Averaging method for F1 ('binary', 'macro', etc.).

    random_state : int, default=42
        Random seed for reproducibility.

    Returns
    -------
    dict
        Dictionary with model, splits, probabilities, and encoder.
    """
    import optuna
    import matplotlib.pyplot as plt
    from lightgbm import LGBMClassifier, early_stopping
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import (
        classification_report, f1_score,
        roc_auc_score, roc_curve, precision_recall_curve
    )

    # ---------------------------------------
    # Preprocessing and Target Encoding
    # ---------------------------------------
    df = df.fillna(0)
    drop_cols = drop_cols or []
    drop_cols = list(set(drop_cols + [target_col, id_col, period_col]))

    X = df.drop(columns=drop_cols)
    y_raw = df[target_col]

    if y_raw.nunique() > 2:
        raise ValueError("Target column must be binary.")

    if y_raw.dtype == 'object' or not set(y_raw.unique()).issubset({0, 1}):
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y_raw == positive_class)
    else:
        label_encoder = None
        y_encoded = (y_raw == positive_class).astype(int)

    period = df[period_col]
    train_mask = period == "train"
    val_mask = period == "validation"
    test_mask = period == "test"

    X_train, y_train = X[train_mask], y_encoded[train_mask]
    X_val, y_val = X[val_mask], y_encoded[val_mask]
    X_test, y_test = X[test_mask], y_encoded[test_mask] if test_mask.any() else (None, None)

    # ---------------------------------------
    # Optuna Hyperparameter Tuning
    # ---------------------------------------
    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "binary_logloss",
            "verbosity": -1,
            "boosting_type": "gbdt",
            "random_state": random_state,
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 15, 100),
            "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
            "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
            "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        }

        model = LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="binary_logloss",
            callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
        )

        preds = model.predict(X_val)
        return f1_score(y_val, preds, average=scoring_metric)

    sampler = optuna.samplers.TPESampler(seed=random_state)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=n_trials)

    # ---------------------------------------
    # Final Model Training
    # ---------------------------------------
    best_params = study.best_trial.params
    best_params["objective"] = "binary"
    final_model = LGBMClassifier(**best_params)
    final_model.fit(X_train, y_train)

    # ---------------------------------------
    # Evaluation on Train/Val/Test Sets
    # ---------------------------------------
    def evaluate_split(X, y, name="Set"):
        y_pred = final_model.predict(X)
        y_proba = final_model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y, y_proba)
        precision, recall, _ = precision_recall_curve(y, y_proba)
        auc = roc_auc_score(y, y_proba)
        print(f"\n📊 {name} Classification Report:")
        print(classification_report(y, y_pred))
        return {
            "y_pred": y_pred,
            "y_proba": y_proba,
            "fpr": fpr,
            "tpr": tpr,
            "precision": precision,
            "recall": recall,
            "auc": auc
        }

    eval_train = evaluate_split(X_train, y_train, name="Train")
    eval_val = evaluate_split(X_val, y_val, name="Validation")
    eval_test = evaluate_split(X_test, y_test, name="Test") if X_test is not None else None

    # ---------------------------------------
    # Plot Curves for Train / Val / Test
    # ---------------------------------------
    fig, axes = plt.subplots(2, 3, figsize=(18, 8))

    # ROC Curves
    axes[0, 0].plot(eval_train["fpr"], eval_train["tpr"], label=f"AUC = {eval_train['auc']:.2f}")
    axes[0, 0].plot([0, 1], [0, 1], linestyle="--", color="gray")
    axes[0, 0].set_title("ROC Curve (Train)")
    axes[0, 0].set_xlabel("FPR")
    axes[0, 0].set_ylabel("TPR")
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(eval_val["fpr"], eval_val["tpr"], label=f"AUC = {eval_val['auc']:.2f}")
    axes[0, 1].plot([0, 1], [0, 1], linestyle="--", color="gray")
    axes[0, 1].set_title("ROC Curve (Validation)")
    axes[0, 1].set_xlabel("FPR")
    axes[0, 1].set_ylabel("TPR")
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    if eval_test:
        axes[0, 2].plot(eval_test["fpr"], eval_test["tpr"], label=f"AUC = {eval_test['auc']:.2f}")
        axes[0, 2].plot([0, 1], [0, 1], linestyle="--", color="gray")
        axes[0, 2].set_title("ROC Curve (Test)")
        axes[0, 2].set_xlabel("FPR")
        axes[0, 2].set_ylabel("TPR")
        axes[0, 2].legend()
        axes[0, 2].grid(True)

    # Precision-Recall Curves
    axes[1, 0].plot(eval_train["recall"], eval_train["precision"])
    axes[1, 0].set_title("Precision-Recall (Train)")
    axes[1, 0].set_xlabel("Recall")
    axes[1, 0].set_ylabel("Precision")
    axes[1, 0].grid(True)

    axes[1, 1].plot(eval_val["recall"], eval_val["precision"])
    axes[1, 1].set_title("Precision-Recall (Validation)")
    axes[1, 1].set_xlabel("Recall")
    axes[1, 1].set_ylabel("Precision")
    axes[1, 1].grid(True)

    if eval_test:
        axes[1, 2].plot(eval_test["recall"], eval_test["precision"])
        axes[1, 2].set_title("Precision-Recall (Test)")
        axes[1, 2].set_xlabel("Recall")
        axes[1, 2].set_ylabel("Precision")
        axes[1, 2].grid(True)

    plt.tight_layout()
    plt.show()

    # ---------------------------------------
    # Return Training Artifacts
    # ---------------------------------------
    return {
        "model": final_model,
        "X_train": X_train,
        "y_train": y_train,
        "X_val": X_val,
        "y_val": y_val,
        "X_test": X_test,
        "y_test": y_test,
        "y_proba_train": eval_train["y_proba"],
        "y_proba_val": eval_val["y_proba"],
        "y_proba_test": eval_test["y_proba"] if eval_test else None,
        "label_encoder": label_encoder,
        "attribute_names": X_train.columns.tolist(),
        "curve_plot": fig
    }


In [ ]:
# this should take ~2 minutes to run!
purchase_prediction_optuna_lightgbm = train_lightgbm_optuna_binary_with_temporal_split(
  purchase_prediction_df
)

## 3.3 Model Comparison: Logistic Regression vs LightGBM

We evaluate two binary classifiers — **Logistic Regression** and **LightGBM** — on the purchase prediction task across the training, validation, and test sets.

---

#### 🔍 Classification Report Summary

| Metric         | Model               | Train Set | Validation Set | Test Set |
|----------------|---------------------|-----------|----------------|----------|
| **Accuracy**   | Logistic Regression | 63%       | 64%            | 64%      |
|                | LightGBM            | 79%       | 91%            | 84%      |
| **Recall**     | Logistic Regression | 59%       | 57%            | 61%      |
|                | LightGBM            | 26%       | 11%            | 13%      |
| **Precision**  | Logistic Regression | 38%       | 8%             | 22%      |
|                | LightGBM            | 90%       | 12%            | 32%      |
| **F1 Score**   | Logistic Regression | 46%       | 14%            | 33%      |
|                | LightGBM            | 40%       | 11%            | 18%      |

---

### ✅ Key Takeaways

#### 1. **Overall Accuracy**
- LightGBM achieves significantly higher accuracy across all sets, especially validation (91%) and test (84%).
- However, high accuracy is **driven by dominance of the majority class (non-purchasers)**.

#### 2. **Recall for Class 1 (Purchasers)**
- Logistic Regression consistently outperforms LightGBM on recall: it identifies more actual purchasers.
- LightGBM misses most purchase cases — recall drops to 11–13% on validation/test.

> 🧠 If recall is critical (e.g. targeting all likely purchasers), Logistic Regression is more suitable.

#### 3. **Precision for Class 1 (Purchasers)**
- Logistic Regression shows low precision (as low as 8% on validation), meaning many false positives.
- LightGBM maintains better precision (up to 32%) by being selective — but at the cost of recall.

> 🎯 If avoiding false positives is the goal (e.g. expensive follow-up), LightGBM may be preferred.

#### 4. **F1 Score for Class 1**
- Logistic Regression has a stronger F1 score on the test set (0.33 vs 0.18), showing more balanced performance.
- LightGBM’s sharp recall drop outweighs its precision advantage.

---

### 🧠 Interpretation

- **Logistic Regression** is more balanced overall and better at surfacing positive cases.
- **LightGBM** is overconfident in class 0 predictions and may require:
  - Threshold tuning
  - Class reweighting or oversampling
  - Custom evaluation metrics (e.g., F1-score or recall for class 1)

---

### 🔭 Next Steps

- Analyze the ROC and Precision-Recall curves to find optimal thresholds.
- Consider **adjusting class weights** or **sampling strategies** to address imbalance.
- Choose a model based on **your business priority**:
  - High recall → Logistic Regression
  - High precision or accuracy → LightGBM (with tuning)

## 3.4 Curve Comparisons

Below we compare the ROC and Precision-Recall curves of both models across **train**, **validation**, and **test** sets.


### 3.4.1 Logistic Regression Curves

In [ ]:
purchase_prediction_logistic_regression["curve_plot"]

### 3.4.2 LightGBM Curves

In [ ]:
purchase_prediction_optuna_lightgbm["curve_plot"]

### 3.4.3 ROC Curves (Discrimination Power)

| Metric         | Logistic Regression | LightGBM      |
|----------------|---------------------|---------------|
| **AUC (Train)**     | 0.66                | **0.86**        |
| **AUC (Validation)**| 0.66                | 0.65            |
| **AUC (Test)**      | 0.66                | 0.65            |

- **LightGBM shows stronger discrimination** on the training set (AUC = 0.86), but this does **not translate to validation/test**.
- Both models generalize similarly on unseen data (AUC ≈ 0.65–0.66), suggesting:
  - LightGBM may be overfitting slightly.
  - Logistic Regression generalizes more consistently.

> 🧠 Interpretation: Logistic Regression is less powerful, but stable. LightGBM has stronger learning capacity, but risks overconfidence without tuning.

---

### 3.4.4 Precision-Recall Curves

- **Train PR Curve**: LightGBM dominates with **high precision across the recall spectrum**, suggesting great ranking capability in-sample.
- **Validation & Test PR Curves**:
  - Both models show **low precision** across the board, especially at moderate recall.
  - Precision drops quickly beyond the top-ranked predictions.

| Behavior                          | Logistic Regression       | LightGBM                     |
|----------------------------------|----------------------------|------------------------------|
| 📉 Precision at High Recall      | Drops sharply              | Drops sharply                |
| 🎯 Precision at Low Recall       | Lower but smoother         | Higher, then steep drop-off  |
| 🧪 Overfitting Signal (PR shape) | More stable across splits  | Train ≫ Validation/Test gap  |

> 📌 Precision-recall curves confirm that **LightGBM is more confident**, but it doesn't rank unseen positives well without tuning.

---

### 3.4.5 Overall Model Summary

| Aspect               | Logistic Regression                     | LightGBM                            |
|----------------------|------------------------------------------|-------------------------------------|
| **Generalization**   | Consistent AUC across splits             | Slight train-overfit on AUC         |
| **Recall for Class 1**| Higher out-of-the-box                   | Very low unless threshold is tuned  |
| **Precision**        | Lower overall, smoother                 | High at top, drops fast             |
| **Recommendation**   | Balanced recall model                   | Calibrate threshold for precision-first use cases |

Possible further next steps might include the following:

- 📉 **Threshold Tuning**: Consider adjusting LightGBM’s decision threshold using PR curve inflection points.
- ⚖️ **Custom Objective**: Use F1-score or recall@k if positive class coverage matters.
- ⚙️ **Resampling or Class Weights**: Especially for LightGBM, to correct its strong bias toward the majority class.

# 4. Model Explanations with SHAP and LIME

Now that we’ve trained our binary classification model, it’s time to move beyond accuracy and performance metrics.

While metrics like F1-score and ROC-AUC are helpful to **quantify** performance, they don’t tell us **why** the model is making a particular prediction — especially on edge cases or misclassifications.

In many real-world applications, especially in regulated or sensitive domains, we must ensure that our models are:
- **Interpretable**: Can a human understand the key factors behind a prediction?
- **Transparent**: Can we explain both correct and incorrect outcomes?
- **Accountable**: Can we defend model behavior if it leads to decisions that affect real users?

To address this, we’ll use the function `generate_binary_classification_model_explanations`, which combines two powerful tools:

**SHAP (SHapley Additive exPlanations)**
- Provides a **global view** of which features are driving model predictions on average.
- Also supports **local explanations** by showing how feature values contribute to a single prediction.

**LIME (Local Interpretable Model-agnostic Explanations)**
- Provides **case-by-case analysis** by fitting interpretable local models.
- In this function, we use LIME to explain:
  - ✅ True Positives
  - ✅ True Negatives
  - ❌ False Positives
  - ❌ False Negatives

This categorization helps us **scrutinize individual decisions**, identify systematic issues, and build trust in the model’s reasoning process.

> ⚠️ Remember: Building an accurate model is great — but not enough. Responsible ML practice requires understanding what’s happening under the hood, especially when deploying into production environments with fairness, compliance, or user trust at stake.


In [ ]:
def generate_binary_classification_model_explanations(model_outputs: dict) -> dict:
    """
    Generate SHAP and LIME explanations for a binary classification model.
    LIME samples include examples from all 4 prediction categories:
    - True Positive, True Negative, False Positive, False Negative

    Parameters:
    - model_outputs: Dictionary with model, data splits, labels, and feature names

    Returns:
    - Dictionary with SHAP and LIME plotting elements (matplotlib Figure objects)
    """
    # --- Local imports ---
    import shap
    import lime.lime_tabular
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd

    # --- Unpack input dictionary ---
    model = model_outputs["model"]
    X_train = model_outputs["X_train"]
    X_val = model_outputs["X_val"]
    y_val = model_outputs["y_val"]
    attribute_names = model_outputs.get("attribute_names", X_train.columns.tolist())

    # Predict on validation set
    y_val_pred = model.predict(X_val)

    # Get class names
    if "label_encoder" in model_outputs and hasattr(model_outputs["label_encoder"], "inverse_transform"):
        class_names = list(model_outputs["label_encoder"].inverse_transform([0, 1]))
    else:
        class_names = ["Class 0", "Class 1"]

    results = {}

    # --- SHAP Explanations ---
    print("🔍 Generating SHAP explanations...")
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_val)

    fig_bar = plt.figure()
    shap.summary_plot(shap_values, X_val, plot_type="bar", show=False)
    plt.title("SHAP Feature Importance (Bar)")
    results["shap_summary_bar_fig"] = fig_bar

    fig_summary = plt.figure()
    shap.summary_plot(shap_values, X_val, show=False)
    plt.title("SHAP Summary Plot")
    results["shap_summary_fig"] = fig_summary

    results["shap_explainer"] = explainer
    results["shap_values"] = shap_values

    # --- LIME Explanations ---
    print("🔍 Generating LIME explanations for TP, TN, FP, FN...")
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values,
        feature_names=attribute_names,
        class_names=class_names,
        mode="classification"
    )

    def predict_proba_with_names(x):
        return model.predict_proba(pd.DataFrame(x, columns=attribute_names))

    lime_figures = []

    # Map each (true, pred) pair to a label
    lime_categories = {
        (1, 1): "True Positive",
        (0, 0): "True Negative",
        (0, 1): "False Positive",
        (1, 0): "False Negative"
    }

    for (true_label, pred_label), category_name in lime_categories.items():
        label_name = class_names[true_label]
        pred_name = class_names[pred_label]
        mask = (y_val == true_label) & (y_val_pred == pred_label)

        if mask.sum() == 0:
            print(f"⚠️ Skipping category {category_name} — no matching examples.")
            continue

        sample_indices = y_val[mask].sample(n=min(3, mask.sum()), random_state=42).index

        print(f"\n📊 LIME: {category_name} (True={label_name}, Pred={pred_name})")

        for idx in sample_indices:
            instance = X_val.loc[idx].values
            exp = lime_explainer.explain_instance(instance, predict_proba_with_names, num_features=20)
            fig = exp.as_pyplot_figure()
            fig.suptitle(
                f"LIME | {category_name} | True={label_name} | Pred={pred_name} | Index={idx}",
                fontsize=12
            )
            lime_figures.append({
                "true_label": int(true_label),
                "predicted_label": int(pred_label),
                "true_label_name": label_name,
                "predicted_label_name": pred_name,
                "category": category_name,
                "index": idx,
                "figure": fig
            })

    results["lime_explanations"] = lime_figures

    print("✅ SHAP and LIME explanations generated.")
    return results


In [ ]:
# this should take ~2 minutes to run!
purchase_prediction_lightgbm_explanations = generate_binary_classification_model_explanations(purchase_prediction_optuna_lightgbm)

## 4.2 SHAP Explanation Insights

The SHAP analysis helps us understand which features most strongly influence the model’s predictions. Here's what we observe from the plots:

### 4.2.1 Feature Importance Bar Plot

This plot shows the **mean absolute SHAP value** for each feature, indicating its average contribution to the model’s output — regardless of direction (positive or negative):

- **Top drivers of model predictions** are:
  - `total_clicks`
  - `avg_session_duration`
  - `impressions_seen`
  - `mobile_usage_minutes`
  - `social_shares`

These are all high-engagement behavioral metrics, suggesting the model is heavily influenced by **user interaction with the platform**.

In [ ]:
purchase_prediction_lightgbm_explanations["shap_summary_bar_fig"]

### 4.2.2 SHAP Summary Plot (Beeswarm)

This plot adds more nuance:
- **Each dot** represents a SHAP value for one row in the validation set.
- **Color** reflects the feature value: blue = low, red = high.
- **Position on the X-axis** shows whether the feature is pushing the prediction **up** (right) or **down** (left).

Key takeaways:
- Users with **high `total_clicks`**, `avg_session_duration`, and `mobile_usage_minutes` (red dots on the right) tend to **increase the model's prediction** (likely toward the positive class).
- Users with **low values** for these features (blue dots on the left) tend to **decrease** the prediction.
- Some features like `prefers_gift_experiences` and `international_tourist` have minor but consistent effects.

### 4.2.3 Interpretation Tip

> SHAP values are **local explanations**: they show the impact of each feature for each individual prediction. The bar chart gives a global view, but the summary plot helps uncover patterns like: “high values of X push predictions up.”

In [ ]:
purchase_prediction_lightgbm_explanations["shap_summary_fig"]

## 4.3 Interpret LIME Bar Charts

LIME (Local Interpretable Model-agnostic Explanations) helps us understand how a model made a **specific prediction** by approximating its behavior locally with an interpretable linear model.

Each LIME explanation is visualized as a **horizontal bar chart**, showing how the input features **push** the prediction toward either class 0 or class 1.

---

### 4.3.1 What the Bar Chart Shows

- Each bar corresponds to a **feature** used in the prediction.
- The **length of the bar** represents the **strength of that feature’s influence** on the prediction.
- The **direction** of the bar indicates **which class** the feature is pushing the prediction toward:
  - 👉 Right (positive weight): pushes toward class 1 (e.g. "Purchased")
  - 👈 Left (negative weight): pushes toward class 0 (e.g. "Not Purchased")

In [ ]:
purchase_prediction_lightgbm_explanations["lime_explanations"][0]["figure"]

In [ ]:
purchase_prediction_lightgbm_explanations["lime_explanations"][3]["figure"]


### 4.3.2 Reading the Chart

- **Top of the chart**: Features with the strongest influence on the model’s decision.
- **Color**: May vary depending on implementation, but the key is always in **bar direction and length**.
- LIME tells a **story for one example** — showing which features made the model more confident in its decision, and which features acted against it.

---

### 4.3.3 Why It Matters

- LIME helps **debug individual decisions** — especially useful for edge cases, errors, or fairness audits.
- Combined with SHAP, it provides a full picture: global feature importance (SHAP) + local decision rationale (LIME).

> Use LIME to ask: "Why did the model make *this* prediction for *this* user?"
